## Data encoding

在拿到Data的時候，我們很難預期全部的欄位都是數字，連續型的為數字，也就是numerical Feature，而另一種就是離散型的Categorical Feature。
電腦只能認得數字(講更嚴謹一點是只認識1跟0)，這時候為了要讓電腦看懂，我們就會對Categorical Feature進行編碼(encoding)。

離散型的資料基本上又分為兩種

有序離散：沒有等級，大小，好壞等邏輯關係的離散資料，Ex:居住地，性別...

無序離散：有前後，大小，好壞等邏輯關係的離散資料：Ex:樓層，學歷(學，碩，博)，能力程度...

來看看一個預測房價的dataset。

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("Housing.csv")
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwaterheating   545 non-null    object
 9   airconditioning   545 non-null    object
 10  parking           545 non-null    int64 
 11  prefarea          545 non-null    object
 12  furnishingstatus  545 non-null    object
dtypes: int64(6), object(7)
memory usage: 55.5+ KB


現實世界的資料往往就如上面的dataset一樣，數字和字串一起組成。

這邊會舉例四種，很常見的categorical feature處理的方式，分別是：
1. Drop Categorical Feature

2. One-hot Encoding

3. Label Encoding

4. Target Encoding

### 1. **Drop Categorical Feature：**

直接把所有Categorical Feature丟棄，只拿Numerical的部分進行訓練跟預測，這邊就不多做示範。

### 2. **One-hot encoding：**

In [25]:
import pandas as pd

df = pd.DataFrame(data={"owner":["a", "b", "c", "d"],
                        "pets":["dog", "mouse", "cat", "cat"],
                        "number":[2, 2, 1, 2]})
df.head()

,owner,pets,number
0,a,dog,2
1,b,mouse,2
2,c,cat,1
3,d,cat,2


In [26]:
from sklearn.preprocessing import OneHotEncoder

onehotencoder = OneHotEncoder()
onehot = onehotencoder.fit_transform(df[['pets']]).toarray()
pd.DataFrame(onehot)

,0,1,2
0,0.0,1.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0


在上面的範例可以看到，one-hot把資料拆解成二元分類，只有0跟1存在，這邊來詳細解釋一下。

本來的pets features有三個為一值，分別為cat, dog, mouse，因此會出現在三個feature columns(0, 1, 2)分別對應著cat, dog, mouse。

如果該owner養的是貓，那該owern的column 0中的數值就是1，其他的column則為0;如果該owner養的是狗，那該owern的column 1中的數值就是1，其他的column則為0，以此類推。

In [11]:
# 查看columns feature的編號(index)
onehotencoder.categories_

[array(['cat', 'dog', 'mouse'], dtype=object)]

In [23]:
onehotencoder.inverse_transform([[0,0,1]])

array([['mouse']], dtype=object)

one-hot encoding適合在無序離散特徵使用，但在實際上Kaggle跟ML的實作中，使用One Hot Encoding要非常的注意，原因很簡單，就是因為它會造成很多的問題。

**問題1. 容易overfitting**

one-hot encoding會大幅增加資料的feature，過多的Feature容易造成模型在訓練的時候overfitting。

**問題2. 造成資料稀疏**

在處理資料時，feature中大量的0，會導致各feature在交錯計算的時候會出現很多的0跟0互相計算，這讓模型參數在跟更新時會經常性的出現無值，會造成參數訓練上的困難。

**問題3. 維度災難**

維度災難用來描述當（數學）空間維度增加時，分析和組織高維空間（通常有成百上千維），因體積指數增加而遇到各種問題場景。這樣的難題在低維空間中不會遇到。one-hot encoding可能會讓本來只10幾個的feature擴展到上百上千的，這很容易導致維度災難。

以上的這些問題導致了使用OneHot時要注意該特徵是否唯一值(unique value)非常多，也很常搭配降維方法一起使用。
另外也要注意使用決策樹模型時不適合使用one-hot encoding。

### 3. **Label Encoding:**

In [27]:
import pandas as pd

df = pd.DataFrame(data={"owner":["a", "b", "c", "d"],
                        "pets":["dog", "mouse", "cat", "cat"],
                        "number":[2, 2, 1, 2]})
df.head()

,owner,pets,number
0,a,dog,2
1,b,mouse,2
2,c,cat,1
3,d,cat,2


In [29]:
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
label = labelencoder.fit_transform(df[['pets']])
pd.DataFrame(label)

/Users/bananaiselite/anaconda3/envs/test/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,0
0,1
1,2
2,0
3,0


In [30]:
# 查看features編碼
labelencoder.classes_

array(['cat', 'dog', 'mouse'], dtype=object)

<img src="pics/one-hot v.s label.png" alt="one-hot v.s label" style="width: 500px"/>

在上面的範例可以看到，label encoding把資料都編上號碼，透過labelencoder.classes_可以知道，cat,dog和mouse分別對應0,1,2，因此可以看到養貓的owner為0，養狗的為1，養老鼠的為2。

Label encoding不適合用於無序性離散資料，因為這個方法會造成feature有排序關係。

舉個例子：寵物種類[dog,cat,dog,mouse,cat]為離散型資料，透過Label encoding轉換為[1,0,1,2,0]。就會產生了一個奇怪的現象：cat和mouse的平均值是dog，所以label encoding最直觀的缺點就是賦值難以解釋，不適合用在無序性離散資料。

### 4. **Target Encoding:**

在某特徵中的唯一值非常多，不適合使用one-hot encoding，如果是無序離散資料，Label encoding也不是很適合，這時候就可以使用target encoding。

target encoding使一種統稱，方法有很多種，只要使用從target提取出來的資訊，來取代類別特徵，這種方法都被稱為Target encoding。
這樣描述有點抽象，我們來看一些例子。

**mean encoding**

In [31]:
import pandas as pd
import numpy as np

df = pd.read_csv("Housing.csv")
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [37]:
# 我們使用furnishingstatus來做舉例
data = df[["furnishingstatus", "price"]]
# 查看furnishingstatus的唯一值
data["furnishingstatus"].unique()

array(['furnished', 'semi-furnished', 'unfurnished'], dtype=object)

In [41]:
# group用法
# 把屬於不同furnishingstatus的price取出
groups = data.groupby("furnishingstatus")["price"]

for g in groups:
    print("furnishingstatus:",g[0])
    print(g[1])
    print("="*40)

furnishingstatus: furnished
0      13300000
1      12250000
3      12215000
4      11410000
8       9870000
         ...   
509     2590000
512     2520000
522     2380000
523     2380000
543     1750000
Name: price, Length: 140, dtype: int64
furnishingstatus: semi-furnished
2      12250000
5      10850000
6      10150000
11      9681000
12      9310000
         ...   
502     2660000
503     2660000
514     2485000
527     2275000
541     1767150
Name: price, Length: 227, dtype: int64
furnishingstatus: unfurnished
7      10150000
9       9800000
16      9100000
21      8680000
28      8400000
         ...   
538     1890000
539     1855000
540     1820000
542     1750000
544     1750000
Name: price, Length: 178, dtype: int64


In [44]:
# 把不同種類furnishingstatus的price轉換成平均值
data["furnishingstatus_mean"] = data.groupby("furnishingstatus")["price"].transform("mean")
data

,furnishingstatus,price,furnishingstatus_mean
0,furnished,13300000,5.495696e+06
1,furnished,12250000,5.495696e+06
2,semi-furnished,12250000,4.907524e+06
3,furnished,12215000,5.495696e+06
4,furnished,11410000,5.495696e+06
...,...,...,...
540,unfurnished,1820000,4.013831e+06
541,semi-furnished,1767150,4.907524e+06
542,unfurnished,1750000,4.013831e+06
543,furnished,1750000,5.495696e+06


Target Encoding的原理是把Categorical Feature跟Target的關係算出來。也是因為這樣，造成在訓練時很容易overfitting，並且因為是取平均，對於極端值很敏感，所以我們通常會使用smooth或是K-fold來避免這種情況。

**K-fold target**

<img src="pics/k-fold 1.webp" alt="k-fold 1" style="width: 700px"/>

<img src="pics/k-fold 2.webp" alt="k-fold 2" style="width: 700px"/>

<img src="pics/k-fold 3.webp" alt="k-fold 3" style="width: 700px"/>

<img src="pics/k-fold 4.webp" alt="k-fold 4" style="width: 700px"/>

<img src="pics/k-fold 5.webp" alt="k-fold 5" style="width: 700px"/>

pic source : https://medium.com/@pouryaayria/k-fold-target-encoding-dfe9a594874b

In [34]:
# KFold用法示範
from sklearn.model_selection import KFold

X = np.array([[1], [2], [3], [4], [5], [6], [7], [8], [9], [10]])
kf = KFold(n_splits=5, shuffle=True, random_state=666)

for train_index, test_index in kf.split(X):

    print(f"Train: index={train_index}")
    print(f"Test:  index={test_index}")

Train: index=[0 1 2 3 4 6 8 9]
Test:  index=[5 7]
Train: index=[1 2 3 4 5 6 7 8]
Test:  index=[0 9]
Train: index=[0 1 2 5 6 7 8 9]
Test:  index=[3 4]
Train: index=[0 2 3 4 5 6 7 9]
Test:  index=[1 8]
Train: index=[0 1 3 4 5 7 8 9]
Test:  index=[2 6]


In [31]:
import pandas as pd
import numpy as np

df = pd.read_csv("Housing.csv")
df.head()

data = df[["furnishingstatus", "price"]]
data

,furnishingstatus,price
0,furnished,13300000
1,furnished,12250000
2,semi-furnished,12250000
3,furnished,12215000
4,furnished,11410000
...,...,...
540,unfurnished,1820000
541,semi-furnished,1767150
542,unfurnished,1750000
543,furnished,1750000


In [49]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=666)

result = pd.DataFrame()
for train_index, test_index in kf.split(data):

    unique_val_mean = data.iloc[train_index].groupby("furnishingstatus")["price"].mean()
    # print(unique_val_mean)

    result = pd.concat([result, pd.DataFrame(unique_val_mean).T])

print(result.mean())

furnishingstatus
furnished         5.494313e+06
semi-furnished    4.907589e+06
unfurnished       4.015086e+06
dtype: float64


在K-fold交叉驗證中，每個fold都有不同的target，這意味著我們可以從多個不同的子集來估計每個類別的target mean。這些均值估計的平均值通常更穩定且不受極端值的影響，從而減少了overfitting的可能性。

這種方法又稱為Leave-One-Out Encoding(LOO Encoding)，LOO Encoding在Category Encoders這個Package裡面有支援，使用方法跟所有sklearn的feature transform一樣。

In [58]:
import category_encoders as ce
import pandas as pd
import numpy as np

df = pd.read_csv("Housing.csv")
df.head()

X = df['furnishingstatus']
y = df['price']

encoder = ce.LeaveOneOutEncoder(cols=["furnishingstatus"])
encoder.fit_transform(X, y)

,furnishingstatus
0,5.439550e+06
1,5.447104e+06
2,4.875035e+06
3,5.447356e+06
4,5.453147e+06
...,...
540,4.026226e+06
541,4.921420e+06
542,4.026621e+06
543,5.522643e+06


**smooth**

方法如下

__*encoding = weight * 類別平均 + (1 - weight) * 整體平均*__

weight 為0~1之間的數值，從類別出現的比率計算而來

公式如下

__*weight = n / (n + m)*__

weight為兩個參數所組成，n 為該類別出現在data的次數，m 則為 smoothing factor，須自行設定，越大的m，代表越會趨向於整體平均。

In [50]:
import pandas as pd
import numpy as np

df = pd.read_csv("Housing.csv")
df.head()

data = df[["furnishingstatus", "price"]]
data

,furnishingstatus,price
0,furnished,13300000
1,furnished,12250000
2,semi-furnished,12250000
3,furnished,12215000
4,furnished,11410000
...,...,...
540,unfurnished,1820000
541,semi-furnished,1767150
542,unfurnished,1750000
543,furnished,1750000


In [57]:
col = 'furnishingstatus'
# 假設m值為2.0
m = 2.0
for val in data[col].unique():
    n = len(data[data[col]==val])/len(data)
    print(val,"的n值：" , n)

    weight = n/(n+m)
    print(val,"的weight：" , weight)

    #encoding = weight * 類別平均 + (1 - weight) * 整體平均
    encoding = weight * data[data[col]==val]["price"].mean() + (1 - weight) * data["price"].mean()
    print(val, "的encoding:", encoding)
    print()

furnished 的n值： 0.25688073394495414
furnished 的weight： 0.11382113821138211
furnished 的encoding: 4849701.073170732

semi-furnished 的n值： 0.41651376146788993
semi-furnished 的weight： 0.17236142748671224
semi-furnished 的encoding: 4790996.871678056

unfurnished 的n值： 0.326605504587156
unfurnished 的weight： 0.14037854889589907
unfurnished 的encoding: 4661038.548895899



**Beta Target Encoding**

BTE會紅起來最主要是在[Kaggle Avito Demand Prediction Challenge](https://www.kaggle.com/c/avito-demand-prediction/overview)中，第14名（top 1%）的選手公布了這個方法，並且顯現了在LightGBM上達到顯著的改善。

由於牽涉的數學式跟知識背景較多，這邊先不介紹，但非常推薦去了解一下這個encoding，是目前很多人會使用的一種Target encoding，連結如下。

[Kaggle Master分享編碼神技-Beta Target Encoding](https://mp.weixin.qq.com/s?__biz=Mzk0NDE5Nzg1Ng==&mid=2247494163&idx=1&sn=5aba9ce08911f12b06619226809cdc7d&chksm=c32af39cf45d7a8aad50b7dfc58dc6eee6128d7f9c0998b86cdfb45f1419427b5ca6a4e0a3ec&mpshare=1&scene=21&srcid=0406aBjMtq5tN6QIki5R1Zj8&sharer_sharetime=1617703540684&sharer_shareid=9c4625e82fc6c8a2cb56fccd24aba0f4#wechat_redirect)

沒有最好的Encoding，每一種Encoding都有自己適合的位子，也有不同的改善空間，如果模型訓練效果不好的話，可以試試看改一種Encoding方法喔。

推薦閱讀：
[也太全了吧！分享16種機器學習類別特徵處理方法！](https://zhuanlan.zhihu.com/p/560004588)